In [14]:
from surprise import Dataset
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import SVDpp, KNNWithMeans
from surprise import accuracy

In [15]:
# Завантаження даних
data = Dataset.load_builtin('ml-100k')

In [16]:
# Поділ на тренувальний і тестовий набори
trainset, testset = train_test_split(data, test_size=0.2)


In [17]:
# Параметри для SVD++
param_grid_svdpp = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

# Налаштування GridSearchCV для SVD++
grid_search_svdpp = GridSearchCV(SVDpp, param_grid_svdpp, measures=['rmse', 'mae'], cv=5)
grid_search_svdpp.fit(data)

# Виведення найкращих параметрів для SVD++
best_params_svdpp = grid_search_svdpp.best_params['rmse']
print("Найкращі параметри для SVD++:", best_params_svdpp)

Найкращі параметри для SVD++: {'n_factors': 50, 'lr_all': 0.01, 'reg_all': 0.05}


In [18]:
# Параметри для KNNWithMeans
param_grid_knn_means = {
    'k': [10, 40],
    'min_k': [1, 3],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

# Налаштування GridSearchCV для KNNWithMeans
grid_search_knn_means = GridSearchCV(KNNWithMeans, param_grid_knn_means, measures=['rmse', 'mae'], cv=5)
grid_search_knn_means.fit(data)

# Виведення найкращих параметрів для KNNWithMeans
best_params_knn_means = grid_search_knn_means.best_params['rmse']
print("Найкращі параметри для KNNWithMeans:", best_params_knn_means)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [28]:
# Оцінка моделей на тестовому наборі
model_svdpp = SVDpp(**best_params_svdpp)
model_svdpp.fit(trainset)

model_knn_means = KNNWithMeans(**best_params_knn_means['sim_options'], k=best_params_knn_means['k'], min_k=best_params_knn_means['min_k'])
model_knn_means.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [20]:
# Прогнозування на тестовому наборі
predictions_svdpp = model_svdpp.test(testset)
predictions_knn_means = model_knn_means.test(testset)


In [21]:
# Оцінка результатів
from surprise import accuracy
rmse_svdpp = accuracy.rmse(predictions_svdpp)
rmse_knn_means = accuracy.rmse(predictions_knn_means)

print(f"SVD++ RMSE: {rmse_svdpp}")
print(f"KNNWithMeans RMSE: {rmse_knn_means}")

RMSE: 0.9157
RMSE: 0.9574
SVD++ RMSE: 0.9156747864469647
KNNWithMeans RMSE: 0.9574106694773667


Після налаштування та оцінки двох моделей — SVD++ та KNNWithMeans — на наборі даних MovieLens 100K, модель SVD++ показала кращі результати за метрикою RMSE (0.9157), що свідчить про точніші прогнози порівняно з KNNWithMeans, яка отримала RMSE 0.9574. Це означає, що SVD++ краще справляється з завданням прогнозування рейтингу фільмів на основі взаємодій користувачів і фільмів у цьому конкретному випадку.

In [45]:
# Створення моделі з використанням SVDpp з найкращими параметрами
algo = SVDpp(n_factors=best_params_svdpp['n_factors'], 
             lr_all=best_params_svdpp['lr_all'], 
             reg_all=best_params_svdpp['reg_all'])
# Навчання моделі
algo.fit(trainset)

# Отримання прогнозів на тестовому наборі
predictions = algo.test(testset)

# Функція для підрахунку кількості рецензій користувача
def count_reviews(user_id, predictions):
    return sum(1 for pred in predictions if pred.uid == str(user_id))

# Функція для отримання топ-10 рекомендацій для користувача
def get_top_n_recommendations(user_id, algo, n=10):
    all_movie_ids = set(iid for (_, iid, _) in testset)
    rated_movie_ids = set(pred.iid for pred in predictions if pred.uid == str(user_id))
    unrated_movie_ids = all_movie_ids - rated_movie_ids
    
    recommendations = []
    for movie_id in unrated_movie_ids:
        predicted_rating = algo.predict(str(user_id), movie_id).est
        recommendations.append((movie_id, predicted_rating))
    
    # Сортування за прогнозованим рейтингом
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:n]

# Кількість рецензій користувача
user_id = 22
num_reviews = count_reviews(user_id, predictions)
print(f"Кількість рецензій користувача {user_id}: {num_reviews}")

# Топ-10 рекомендованих фільмів
top_10_movies = get_top_n_recommendations(user_id, algo)

print("Топ-10 фільмів, рекомендованих для користувача:")
for movie_id, predicted_rating in top_10_movies:
    print(f"Фільм {movie_id} з прогнозованим рейтингом {predicted_rating:.2f}")

Кількість рецензій користувача 22: 27
Топ-10 фільмів, рекомендованих для користувача:
Фільм 174 з прогнозованим рейтингом 5.00
Фільм 173 з прогнозованим рейтингом 5.00
Фільм 172 з прогнозованим рейтингом 5.00
Фільм 50 з прогнозованим рейтингом 5.00
Фільм 168 з прогнозованим рейтингом 4.96
Фільм 181 з прогнозованим рейтингом 4.90
Фільм 483 з прогнозованим рейтингом 4.87
Фільм 127 з прогнозованим рейтингом 4.85
Фільм 187 з прогнозованим рейтингом 4.75
Фільм 12 з прогнозованим рейтингом 4.68
